**Import libraries**

In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

from torchvision import transforms

from tqdm import tqdm

In [2]:
# import wandb
# wandb.init(project="aml-lab03", entity="peiro98", name="aml-lab03")

**Set Arguments**

In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"  # 'cuda' or 'cpu'

# available classes: "dog", "elephant", "giraffe", "guitar", "horse", "house", "person"
NUM_CLASSES = 7

BATCH_SIZE = 4
LR = 5e-4
MOMENTUM = 0.9
WEIGHT_DECAY = 5e-5

NUM_EPOCHS = 25
STEP_SIZE = 10
GAMMA = 0.1

LOG_FREQUENCY = 10
TRAIN_RATIO = 0.75

**Define Data Preprocessing**

In [4]:
# Define transforms for training phase
train_transform = transforms.Compose(
    [
        # 227x227 -> 224x224
        transforms.CenterCrop(224),
        # convert to tensor
        transforms.ToTensor(),
        # normalizes tensor with mean and standard deviation
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

# Define transforms for the evaluation phase
eval_transform = train_transform

### Dataset loading

#### Source dataset

In [5]:
DATA_DIR = "PACS/data"
SRC_DATA_DIR = f"{DATA_DIR}/photo/"

from PACS.PACS import PACSDataset

# Prepare Pytorch train/test Datasets
src_dataset = PACSDataset(SRC_DATA_DIR, transform=train_transform)

print("[SRC] train size:", len(src_dataset))

[SRC] train size: 1670


#### Target dataset

In [6]:
TARGET_DATA_DIR = f"{DATA_DIR}/art_painting/"

# Prepare Pytorch train/test Datasets
target_dataset = PACSDataset(TARGET_DATA_DIR, transform=train_transform)

# take the indicies corresponding to train samples
target_train_indices = np.random.choice(
    len(target_dataset), size=int(TRAIN_RATIO * len(target_dataset))
)
target_val_indices = np.setdiff1d(np.arange(len(target_dataset)), target_train_indices)

target_train_dataset = Subset(target_dataset, target_train_indices)
target_validation_dataset = Subset(target_dataset, target_val_indices)

print("[TARGET] train size:", len(target_train_dataset))
print("[TARGET] validation size:", len(target_validation_dataset))

[TARGET] train size: 1536
[TARGET] validation size: 994


**Prepare Dataloaders**

In [7]:
# Train dataloaders
src_dataloader = DataLoader(
    src_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True
)
# src_train_dataloader = DataLoader(src_train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
# target_train_dataloader = DataLoader(target_train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)

# Validation dataloaders
target_dataloader = DataLoader(
    target_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, drop_last=True
)
# src_validation_dataloader = DataLoader(src_validation_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, drop_last=True)
# target_validation_dataloader = DataLoader(target_validation_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, drop_last=True)

**Prepare Network**

In [8]:
from DANN import build_model as build_DANN

DANN = build_DANN(7, 2, True)  # n. of classes, n. of domains, pretrained

**Prepare Training**

In [9]:
# classification => cross entropy loss
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(
    DANN.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY
)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

## Train w/o domain adaptation

In [10]:
def compute_accuracy(dataloader):
    DANN.train(False)

    # predictions_table = wandb.Table(columns=["image", "label", "preds"])

    running_corrects = 0
    for data, labels in dataloader:
        data = images.to(DEVICE)
        labels = labels.to(DEVICE)

        # Forward Pass
        outputs = DANN(data, True) # True for classifier

        # Get predictions
        _, preds = torch.max(outputs.data, 1)

        # Update Corrects
        running_corrects += torch.sum(preds == labels.data).data.item()

        # if i == 0:
        #     for image, label, pred in zip(images, labels, preds):
        #         predictions_table.add_data(wandb.Image(image), label, pred)
                
    return running_corrects / float(len(target_dataset))


In [11]:
def compute_classification_loss(dataloader):
    criterion = nn.CrossEntropyLoss(reduction='sum')
    DANN.train(False)
    
    loss, n = 0, 0
    for data, labels in dataloader:
        data = images.to(DEVICE)
        labels = labels.to(DEVICE)

        # Forward Pass
        outputs = DANN(data, True) # True for classifier

        # compute the loss
        loss += criterion(outputs.detach(), labels)
        n = n + len(labels)
       
    return loss / n


In [ ]:
DANN = DANN.to(DEVICE)  # this will bring the network to GPU if DEVICE is cuda
# wandb.watch(DANN, log_freq=10)

cudnn.benchmark = True

current_step = 0

for epoch in tqdm(range(NUM_EPOCHS)):
    epoch_loss = 0

    # iterate over the dataset
    for images, labels in src_dataloader:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        DANN.train()

        # zero the gradient
        optimizer.zero_grad()
        outputs = DANN(images, True)

        # compute the loss
        loss = criterion(outputs, labels)

        # compute and propagate the gradient
        loss.backward()
        optimizer.step()

        current_step += 1

        epoch_loss += loss

    source_loss = epoch_loss / len(src_dataloader)
    print(f"[SOURCE]: (avg) loss is {source_loss:.3f}")

    target_loss = compute_classification_loss(target_dataloader)
    target_accuracy = compute_accuracy(target_dataloader)
    print(f"[TARGET]: loss is {target_loss:.3f}, accuracy is {target_accuracy:.3f}")
    
    # compute the average epoch loss
    # wandb.log({"training-loss": training_loss})

    # at the end of each epoch compute the accuracy on the validation accuracy
    # wandb.log({"validation-accuracy": validation_loss})

    # Step the scheduler
    scheduler.step()

## Train w/ domain adaptation

In [11]:
DANN = DANN.to(DEVICE)  # this will bring the network to GPU if DEVICE is cuda
# wandb.watch(DANN, log_freq=10)

cudnn.benchmark = True

current_step = 0

domain_classifier_criterion = nn.BCELoss()

for epoch in tqdm(range(NUM_EPOCHS)):
    epoch_loss = 0

    DANN.train(True)

    # iterate over the dataset
    for (source_images, source_labels), (target_images, _) in zip(src_dataloader, target_dataloader):
        source_images = source_images.to(DEVICE)
        target_images = target_images.to(DEVICE)
        source_labels = source_labels.to(DEVICE)

        # zero the gradient
        optimizer.zero_grad()

        # compute the classifier output for SOURCE images
        classifier_source_outputs = DANN(source_images, True)
        domain_classifier_source_outputs = DANN(source_images, False)
        domain_classifier_target_outputs = DANN(target_images, False)

        # compute the loss for the classifier
        loss = criterion(classifier_source_outputs, source_labels) # supervised task
        loss += criterion(domain_classifier_source_outputs, torch.full((BATCH_SIZE, ), 1, device=DEVICE))
        loss += criterion(domain_classifier_source_outputs, torch.full((BATCH_SIZE, ), 0, device=DEVICE))

        # compute and propagate the gradient
        loss.backward()
        optimizer.step()

        current_step += 1

        epoch_loss += loss

    source_loss = epoch_loss / len(src_dataloader)
    print(f"[SOURCE]: (avg) loss is {source_loss:.3f}")

    target_loss = compute_classification_loss(target_dataloader)
    target_accuracy = compute_accuracy(target_dataloader)
    print(f"[TARGET]: loss is {target_loss:.3f}, accuracy is {target_accuracy:.3f}")
    
    # compute the average epoch loss
    # wandb.log({"training-loss": training_loss})

    # at the end of each epoch compute the accuracy on the validation accuracy
    # wandb.log({"validation-accuracy": validation_loss})

    # Step the scheduler
    scheduler.step()

  0%|          | 0/25 [00:01<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 144.00 MiB (GPU 0; 1.96 GiB total capacity; 1023.27 MiB already allocated; 58.88 MiB free; 1.15 GiB reserved in total by PyTorch)

In [18]:
source_labels.shape

torch.Size([16])